In [ ]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
# import snap
from collections import defaultdict
import random
import playlist
reload(playlist)

In [ ]:
playlist_list = playlist.get_playlist_list()

In [ ]:
len(playlist_list)

In [ ]:
import sys
sys.getsizeof(playlist_list)

In [ ]:
# tracks = get_tracks(data['playlists'][0])
# tracks

In [ ]:
def highest_visited_random_walk_playlist(query,g,dropped_track_id):
#     do not go from query to dropped_track_id
#     or dropped_track_id to query
    
    N = 10000
#     totSteps is the number of total steps where step is going from playlist to track to playlist
    totSteps = 0
#     nHighVisited is the number of track nodes that have been visited at least nv times
    nHighVisited = 0
    nv = 5
#     nt is the least number of tracks which have been visited at least nv times
    nt = 200
    num_visits = defaultdict(int)

    # BUG: was or nHighVisited > nt
    while totSteps < N and nHighVisited < nt:
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            # list() is needed so graph is not changed by remove
            edges = list(g[currPlaylist])
            if currPlaylist == query:
                edges.remove(dropped_track_id)
            currTrack = random.choice(edges)

            # takes 1 step (from a track to playlist)
            edges = list(g[currTrack])
            if currTrack == dropped_track_id:
                edges.remove(query)
            currPlaylist = random.choice(edges)
            num_visits[currTrack] += 1
            if num_visits[currTrack] == nv:
                nHighVisited += 1
        totSteps += currSteps
    return num_visits

In [ ]:
# g, tracks_dict, tracks_id_dict = playlist.make_graph(playlist_list)
g, tracks_dict, tracks_id_dict = playlist.make_graph_dict(playlist_list)

In [ ]:
def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_id = random.choice(tuple(g[playlist_id]))

    if len(g[playlist_id]) < 5:
        return None, None
    num_visits = highest_visited_random_walk_playlist(playlist_id, g, dropped_track_id)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

In [ ]:
# all_true_ids, all_suggested_ids = cal_most_visited(playlist_list)
all_true_ids, all_suggested_ids = playlist.process_playlists(cal_most_visited_playlist, playlist_list,
                                                             multiprocess=False)


In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids)
filename = 'results%d_highest_visited_random_walk_10k.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))

In [ ]:
# np.savez_compressed('basic_random_walk_50k_500_result_new.npz',all_true_ids = all_true_ids, 
#                    all_suggested_ids = all_suggested_ids)

In [ ]:
# %%time
# data = np.load('basic_random_walk_50k_500_result.npz', mmap_mode='r')
# all_true_ids = data['all_true_ids'] 
# all_suggested_ids = data['all_suggested_ids']

In [ ]:
# %%time
# tracks_dict, tracks_id_dict = playlist.make_tracks_dict(playlist_list)

In [ ]:
# max(num_visits.values())

In [ ]:
list(sorted(num_visits, key=num_visits.get, reverse=True)).index(1000)

In [ ]:
# avg_precision = playlist.cal_ave_precision(500, all_true_ids, all_suggested_ids)
# print avg_precision

In [ ]:
# binary_ndcg = playlist.cal_ndcg(all_true_ids, all_suggested_ids, gain=playlist.binary_get_gain)
# print binary_ndcg

In [ ]:
# artist_get_gain = playlist.artist_get_gain(playlist_list, tracks_id_dict)

In [ ]:
# artist_ndcg = playlist.cal_ndcg(all_true_ids, all_suggested_ids, gain=artist_get_gain)
# print artist_ndcg

In [ ]:
# float(False)

In [ ]:
# import pdb; pdb.pm()